In [1]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
import sys
sys.path.append("../../")
from src.config import Configuration

config = Configuration()

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.robot import RAGRobot
from src.service.provider import ProviderService
from langsmith.run_helpers import traceable

config.enable_tracing(project="LEARN")
provider = ProviderService()
robot = RAGRobot(provider=provider)


@traceable(tags=['route'])
def run_full_chain(q):
    a = robot.main_chain.invoke({"question": q})
    return a


From (44, 5) to 44
From (44, 4) to 44


In [4]:
type (robot.major_rag.chain)

langchain_core.runnables.base.RunnableSequence

In [3]:
q = "sinh viên tốt nghiệp ngành kỹ thuật hóa học có thể ứng tuyển vào vị trí nào?"
a = robot.router.invoke({"question": q})
a

'Local'

In [4]:
q = "sinh viên tốt nghiệp ngành công nghệ phần mềm có thể ứng tuyển vào vị trí nào?"
a = robot.router.invoke({"question": q})
a

'Major'

In [5]:
q = "trường có dạy AI không ạ?"
a = robot.router.invoke({"question": q})
a

'Major'

In [6]:
q = "trường có khuôn viên như thế nào?"
a = robot.router.invoke({"question": q})
a

'Other'

In [8]:
a = robot.followup.chain.invoke({"question": q})
a

AIMessage(content='- Chuyên viên vận hành và quản lý sản xuất\n- Chuyên viên nghiên cứu phát triển sản phẩm\n- Chuyên viên tư vấn quản lí và chuyển giao công nghệ\n- Nhân viên kinh doanh hóa chất\n- Nhân viên kiểm soát chất lượng sản phẩm')

## Server RAG

In [3]:
from src.rag.hybrid_rag import HybridGeminiRAG
from src.service.provider import ProviderService
services = ProviderService()

major = services.get_categories().major

rag = HybridGeminiRAG(
    vector_es_index="labse-major", 
    txt_es_index="text-split-major", 
    db_category=major, provider=services)

From (44, 4) to 44


## Local Follow Up

In [4]:
from src.prepare.data_load import DocDataLoader

doc_loader = DocDataLoader()
docs = doc_loader.load_major_docs_full()
d1 = docs[15]
d1.page_content[:30]

From (44, 4) to 44


'Kỹ thuật hoá học là ngành học'

In [5]:
from src.chain.followup import LocalFollowUpChain
followup = LocalFollowUpChain(config=config, doc_content=d1.page_content)

## Router

In [6]:
from langchain_core.runnables import RunnableBranch


branch = RunnableBranch(
    (lambda x: "local" in x["topic"].lower(), followup.chain),
    rag.chain
)


In [7]:
llm = config.get_gemini_pro(convert_system_message=False)
template = """Given the user question below, classify it as either being about `Local`, or `Other`. Known that:
```
    Local: when the question related to "ngành kỹ thuật hóa học" or "ngành kỹ thuật hóa học trường Tôn Đức Thắng".
```
Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
chain = (
    PromptTemplate.from_template(template=template)
    | llm
    | StrOutputParser()
)

In [8]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [9]:
config.enable_tracing(project="LEARN")

In [10]:
from langsmith.run_helpers import traceable

@traceable(tags=['route'])
def run_full_chain(q):
    a = full_chain.invoke({"question": q})
    return a

In [11]:
q = "sinh viên tốt nghiệp ngành kỹ thuật hóa học có thể ứng tuyển vào vị trí nào?"
a = run_full_chain(q)
a

AIMessage(content='- Chuyên viên vận hành và quản lý sản xuất\n- Chuyên viên nghiên cứu phát triển sản phẩm\n- Chuyên viên tư vấn quản lí và chuyển giao công nghệ\n- Nhân viên kinh doanh hóa chất\n- Nhân viên kiểm soát chất lượng sản phẩm')

In [12]:
q = "sinh viên tốt nghiệp ngành công nghệ phần mềm có thể ứng tuyển vào vị trí nào?"
a = run_full_chain(q)
a

'Chuyên viên phân tích yêu cầu (BA), Kiến trúc sư phần mềm (SA), Lập trình viên (Developer), Chuyên viên thử nghiệm phần mềm (Tester), chuyên viên cầu nối (BrSE), chuyên viên nghiên cứu và phát triển ứng dụng (R&D), giảng viên giảng dạy các môn học về ngành kỹ thuật phần mềm'